In [5]:
# k-nearest neighbors on the Iris Flowers Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import numpy as np
# Load a CSV file and drop 'timeSignature' column
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        header = next(csv_reader, None)  # Read the header
        if 'timeSignature' in header:
          index = header.index('timeSignature')
        for row in csv_reader:
             if not row:
                continue
             del row[index]
             dataset.append(row)
    dataset = dataset[1:]
    return dataset


# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])-1):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset):
    # Exclude the last column when calculating mean and standard deviation
    dataset_array = np.array(dataset)
    #print(dataset_array)
    # Exclude the last column when calculating mean and standard deviation
    means = np.mean(dataset_array[:, :-1].astype(float), axis=0)
    stds = np.std(dataset_array[:, :-1].astype(float), axis=0)

    # Apply standard deviation normalization
    for row in dataset:
        for i in range(len(row)-1):  # Exclude the last column
            row[i] = (row[i] - means[i]) / stds[i]

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for _ in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction

# kNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors):
	predictions = list()
	for row in test:
		output = predict_classification(train, row, num_neighbors)
		predictions.append(output)
	return(predictions)

seed(1)
filename = 'final_train.csv'
dataset = load_csv(filename)
# for row in dataset:
#   print(row)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
#minimax = dataset_minmax(dataset)
normalize_dataset(dataset)

# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
num_neighbors = 5
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [67.9054054054054, 64.52702702702703, 66.8918918918919, 68.91891891891892, 67.22972972972973]
Mean Accuracy: 67.095%


In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

def load_dataset(csv_file):
    return pd.read_csv(csv_file)

def prepare_data(df, target_column):
    X = df.drop(columns=[target_column]).values  # Features
    y = df[target_column].values  # Class labels
    return X, y

def knn_classification(X, y, k_neighbors=5, n_splits=5):

    from sklearn.model_selection import cross_val_score, KFold
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.preprocessing import StandardScaler

    # Initialize KNN classifier
    knn = KNeighborsClassifier(n_neighbors=k_neighbors)

    # Scale the features using StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Define cross-validation
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform cross-validation and calculate accuracy
    cv_scores = cross_val_score(knn, X_scaled, y, cv=kf, scoring='accuracy')

    # Calculate mean accuracy
    mean_accuracy = cv_scores.mean()

    return mean_accuracy

# Example usage:
csv_file = "final_train.csv"  # Replace with the path to your CSV file
target_column = "mood"  # Replace with the name of your class label column

# Load dataset
df = load_dataset(csv_file)

# Prepare features (X) and class labels (y)
X, y = prepare_data(df, target_column)

# Perform KNN classification with 5-fold cross-validation
mean_accuracy = knn_classification(X, y)

print("Mean Accuracy:", mean_accuracy*100, "%")


Mean Accuracy: 67.31845481845482 %
